# Showcase: CultPass

## Testing Toolkit

In [10]:
from pathlib import Path
import shutil
import pandas as pd
import os


def rollback_databases() -> None:
    """Delete current DBs and copy backups into place."""

    repo_root = Path(".").resolve()

    cultpass_dst = repo_root / "data" / "external" / "cultpass.db"
    udahub_dst = repo_root / "data" / "core" / "udahub.db"

    backup_dir = repo_root / "data" / "backup"
    cultpass_src = backup_dir / "cultpass.db"
    udahub_src = backup_dir / "udahub.db"

    if not cultpass_src.exists() or not udahub_src.exists():
        raise FileNotFoundError(
            "Missing backup DB(s) in starter/data/backup. "
            "Expected cultpass.db and udahub.db."
        )

    cultpass_dst.parent.mkdir(parents=True, exist_ok=True)
    udahub_dst.parent.mkdir(parents=True, exist_ok=True)

    if cultpass_dst.exists():
        cultpass_dst.unlink()
    if udahub_dst.exists():
        udahub_dst.unlink()

    shutil.copyfile(cultpass_src, cultpass_dst)
    shutil.copyfile(udahub_src, udahub_dst)


def cultpass_db(query: str) -> pd.DataFrame:
    return pd.read_sql_query(
        query, con="sqlite:///./data/external/cultpass.db"
    )


def udahub_db(query: str) -> pd.DataFrame:
    return pd.read_sql_query(
        query, con="sqlite:///./data/core/udahub.db"
    )

## Imports

In [22]:
import sys
from pathlib import Path

# Add the parent directory to Python path to import starter module
sys.path.insert(0, str(Path().resolve().parent))

from starter.agentic.udahub import UdaHubAgent, McpServerList
from langchain_mcp_adapters.client import StreamableHttpConnection
from starter.agentic.udahub import FAQ_AGENT, RESERVATION_AGENT, SUBSCRIPTION_AGENT, BROWSING_AGENT, UdaHubAgent
from dotenv import load_dotenv
from rich.pretty import Pretty

import chromadb
import asyncio

load_dotenv()

True

## Setup


**Reminder**: Remember to run the MCP servers before executing the rest of this notebook:

```bash
python -m starter.mcp_servers.udahub_mcp
python -m starter.mcp_servers.knowledgebase_mcp
python -m starter.mcp_servers.cultpass_mcp
```

### MCP Server Connections

First we set up the connections to the MCP servers that our agent will interact with.
In our showcase for CultPass Card, this is only one MCP server, whichs lets us interact with the CultPass Card system.

In [3]:
mcp_servers = McpServerList().add_connection(
    "cultpass",
    StreamableHttpConnection(
        url="http://localhost:8003/mcp", transport="streamable_http"
    ),
)

### Agent Selection

UDA-Hub comes with a predefined set of agents and its up to the customer to decide which ones should be available in the interaction of UDA-Hub and their users.
Per default all agents are included, but for the sake of demonstration we will manually select them and plug them into the UDA-Hub agent.

In [4]:
showcase_agents = [
    FAQ_AGENT,
    RESERVATION_AGENT,
    SUBSCRIPTION_AGENT,
    BROWSING_AGENT,
]

### Composing UDA-Hub Chat Agent

In the following we will write a fundtion that returns a new UDA-Hub Chat Agent, so that we can switch sessions when needed.

In [5]:
def uda_hub_chat_agent():
    return UdaHubAgent(
        mcp_servers=mcp_servers, 
        agents=showcase_agents, 
        openai_model="gpt-4.1",
    )

## Database Inspection

First up, in order to understand what the agent is doing we want to have a brief look at the current situation of the CultPass database.

### CultPass Experiences

We can see that CultPass Card currently offers 30 different experiences around the world.

Notice that some experiences are marked as premium, meaning that only users with a premium subscription can book these experiences.

In [7]:
cultpass_db("experiences")

,experience_id,title,description,location,when,slots_available,is_premium,created_at,updated_at
0,a6cd9a,Carnival History Tour in Olinda,Discover the origins and vibrant traditions of...,"Pernambuco, Brazil",2026-02-08 03:23:12.544191,21,1,2026-02-07 02:23:12,2026-02-07 02:23:12
1,b99f9b,Sunset Paddleboarding,Glide across calm waters at golden hour with a...,"Santa Catarina, Brazil",2026-02-09 03:23:12.548619,6,0,2026-02-07 02:23:12,2026-02-07 02:23:12
2,68ea3d,Pelourinho Colonial Walk,Wander through colorful streets and learn abou...,"Bahia, Brazil",2026-02-10 03:23:12.548674,16,1,2026-02-07 02:23:12,2026-02-07 02:23:12
3,a5775f,Samba Night at Lapa,Dance the night away at a traditional samba cl...,"Rio de Janeiro, Brazil",2026-02-11 03:23:12.548716,29,0,2026-02-07 02:23:12,2026-02-07 02:23:12
4,fe06ac,Christ the Redeemer Experience,Take a guided trip to one of the New Seven Won...,"Rio de Janeiro, Brazil",2026-02-12 03:23:12.548757,29,1,2026-02-07 02:23:12,2026-02-07 02:23:12
5,139a8a,Modern Art at MASP,Enjoy a guided visit to the São Paulo Museum o...,"São Paulo, Brazil",2026-02-13 03:23:12.548800,29,0,2026-02-07 02:23:12,2026-02-07 02:23:12
6,f217d2,Ibirapuera Park Bike Ride,Cycle through the city's most iconic park with...,"São Paulo, Brazil",2026-02-14 03:23:12.548844,5,1,2026-02-07 02:23:12,2026-02-07 02:23:12
7,72b489,Guided Tour through Vienna,Explore Vienna's rich history and culture with...,"Vienna, Austria",2026-02-15 03:23:12.548884,30,0,2026-02-07 02:23:12,2026-02-07 02:23:12
8,966c93,Danube River Cruise,Experience the beauty of Vienna from the Danub...,"Vienna, Austria",2026-02-16 03:23:12.548925,3,1,2026-02-07 02:23:12,2026-02-07 02:23:12
9,8ad50e,Viennese Coffeehouse Experience,Discover the traditional Viennese coffee cultu...,"Vienna, Austria",2026-02-17 03:23:12.548967,20,0,2026-02-07 02:23:12,2026-02-07 02:23:12


### CultPass Users

Inspecting the userbase of CultPass Card we can see that there are currently 6 users registered.

Notice that some users are blocked, some cancelled their subscription and some chose to upgrade their subscription tier to premium.

In [16]:
cultpass_db("SELECT * FROM users LEFT JOIN subscriptions ON users.user_id = subscriptions.user_id")

,user_id,full_name,email,is_blocked,created_at,updated_at,subscription_id,user_id,status,tier,monthly_quota,started_at,ended_at,created_at,updated_at
0,a4ab87,Alice Kingsley,alice.kingsley@wonderland.com,1,2026-02-07 03:23:12.609707,2026-02-07 02:23:12,38f49f,a4ab87,active,premium,3,2026-02-07 03:23:12.639001,None,2026-02-07 02:23:12,2026-02-07 02:23:12
1,f556c0,Bob Stone,bob.stone@granite.com,0,2026-02-07 03:23:12.609766,2026-02-07 02:23:12,1a9479,f556c0,active,basic,9,2026-02-07 03:23:12.639081,None,2026-02-07 02:23:12,2026-02-07 02:23:12
2,88382b,Cathy Bloom,cathy.bloom@florals.org,0,2026-02-07 03:23:12.609775,2026-02-07 02:23:12,57feda,88382b,active,premium,2,2026-02-07 03:23:12.639123,None,2026-02-07 02:23:12,2026-02-07 02:23:12
3,888fb2,David Noir,david.noir@shadowmail.com,1,2026-02-07 03:23:12.609780,2026-02-07 02:23:12,d3e593,888fb2,active,basic,9,2026-02-07 03:23:12.639163,None,2026-02-07 02:23:12,2026-02-07 02:23:12
4,f1f10d,Eva Green,eva.green@ecosoul.net,0,2026-02-07 03:23:12.609785,2026-02-07 02:23:12,0a2555,f1f10d,cancelled,basic,8,2026-02-07 03:23:12.639203,None,2026-02-07 02:23:12,2026-02-07 02:23:12
5,e6376d,Frank Ocean,frank.ocean@seawaves.io,0,2026-02-07 03:23:12.609789,2026-02-07 02:23:12,be6c45,e6376d,active,basic,9,2026-02-07 03:23:12.639243,None,2026-02-07 02:23:12,2026-02-07 02:23:12


### CultPass Reservations

In [19]:
cultpass_db("SELECT * FROM reservations LEFT JOIN users ON reservations.user_id = users.user_id")

,reservation_id,user_id,experience_id,status,created_at,updated_at,user_id,full_name,email,is_blocked,created_at,updated_at
0,802eff,a4ab87,c47554,reserved,2026-02-07 02:23:12,2026-02-07 02:23:12,a4ab87,Alice Kingsley,alice.kingsley@wonderland.com,1,2026-02-07 03:23:12.609707,2026-02-07 02:23:12
1,0126e1,a4ab87,b99f9b,reserved,2026-02-07 02:23:12,2026-02-07 02:23:12,a4ab87,Alice Kingsley,alice.kingsley@wonderland.com,1,2026-02-07 03:23:12.609707,2026-02-07 02:23:12


### Uda-Hub Accounts

From the accounts table we can see that only CultPass Card is currently connected to UDA-Hub.

In [9]:
udahub_db("accounts")

,account_id,account_name,account_description,created_at,updated_at
0,cultpass,CultPass Card,CultPass Card is a digital membership card tha...,2026-02-07 02:23:40,2026-02-07 02:23:40


### UDA-Hub Users

Furthermore we see, that only one user of CultPass Card, namely Alice Kingsley, so far interacted with UDA-Hub.

In [8]:
udahub_db("users")

,user_id,account_id,external_user_id,user_name,created_at,updated_at
0,662dfd3a-b000-49c1-b4c2-5259a3558daf,cultpass,a4ab87,Alice Kingsley,2026-02-07 02:23:40,2026-02-07 02:23:40


### UDA Hub Knowledge

Inspecting the knowledge table of UDA-Hub database we can see that there are currently 14 enties stored for the usage of FAQs.

In [20]:
udahub_db("SELECT * FROM knowledge")

,article_id,account_id,title,content,tags,created_at,updated_at
0,a019341c-643a-4a17-95fe-2623a38cadaf,cultpass,How to Reserve a Spot for an Event,If a user asks how to reserve an event:\n\n- G...,"reservation, events, booking, attendance",2026-02-07 02:23:40,2026-02-07 02:23:40
1,8907406f-9f93-474e-ac48-7a910675dda9,cultpass,What's Included in a CultPass Subscription,Each user is entitled to 4 cultural experience...,"subscription, benefits, pricing, access",2026-02-07 02:23:40,2026-02-07 02:23:40
2,7cf2f635-ca82-4d96-b32b-4307063eaf54,cultpass,How to Cancel or Pause a Subscription,Users can manage their subscription via the ap...,"cancelation, pause, subscription, billing",2026-02-07 02:23:40,2026-02-07 02:23:40
3,07e2a0fd-5770-49a2-8270-da2301c1e86c,cultpass,How to Handle Login Issues?,Most login issues are resolved with password r...,"login, password, access, escalation",2026-02-07 02:23:40,2026-02-07 02:23:40
4,a25e7353-7645-438b-86a1-34a0ba33131c,cultpass,How to Update Payment Information?,Users can update their payment details through...,"payment, billing, update, support",2026-02-07 02:23:40,2026-02-07 02:23:40
5,0f3dee61-c43f-4a26-88d1-407cb1651ccb,cultpass,Can I cancel my reservation?,Reservations can be canceled up to 24 hours be...,"reservation, cancellation, policy, refund",2026-02-07 02:23:40,2026-02-07 02:23:40
6,ae19e9df-bafb-4375-a1be-1474cfe9e149,cultpass,How do I change my password?,To change your password:\n\n- Open the CultPas...,"password, security, account, update",2026-02-07 02:23:40,2026-02-07 02:23:40
7,4b0697b5-8bb4-4486-a914-5aaffc2417ff,cultpass,Do I get a confirmation email after reserving ...,"Yes, you will receive a confirmation email aft...","confirmation, email, reservation, support",2026-02-07 02:23:40,2026-02-07 02:23:40
8,5d80ae55-75c6-4a89-905f-82bf58391274,cultpass,Can I get more than 4 experiences per month?,The standard CultPass subscription includes 4 ...,"subscription, additional, purchase, events",2026-02-07 02:23:40,2026-02-07 02:23:40
9,bcd6e4b1-f32c-49ed-87ea-ee392b268459,cultpass,Where do you offer experiences?,CultPass offers experiences in various cities ...,"locations, availability, events, expansion",2026-02-07 02:23:40,2026-02-07 02:23:40


## ChromaDB Knowledge Base

As outlined in the README.md, our agents are not going to use the relational databases for extracting knowledge or the experiences of CultPass Card.
Instead, we will use a vector database powered by ChromaDB that contains the experiences of CultPass Card as well as the knowloedge from UDA-Hubs knowledge table.


**Hint**: The knwoledge is synchronized upon starting a new run through the workflow graph, so that the ChromaDB is always up-to-date.

In [26]:
chroma_client = chromadb.PersistentClient(path="../chroma_data")
Pretty(chroma_client.list_collections())

[Collection(name=cultpass), Collection(name=udahub)]

As we can see, there are two collections in the ChromaDB. One for the CultPass experiences and one for the UDA-Hub knowledge base.

### UDA-Hub Collection

The `udahub` collection contains the same information as the relational database table above. The big difference is that the entries are vectorized and can be used for similarity search.

In [32]:
Pretty(chroma_client.get_collection("udahub").get(limit=3))

{
    'ids': [
        'knowledge_a019341c-643a-4a17-95fe-2623a38cadaf',
        'knowledge_8907406f-9f93-474e-ac48-7a910675dda9',
        'knowledge_7cf2f635-ca82-4d96-b32b-4307063eaf54'
    ],
    'embeddings': None,
    'documents': [
        'If a user asks how to reserve an event:\n\n- Guide them to the CultPass app\n- Instruct them to browse the
experience catalog and tap \'Reserve\'\n- If it\'s a premium or limited event, check if reservation confirmation is
required via email\n- Remind them to arrive at least 15 minutes early with their QR code visible\n\n**Suggested 
phrasing:**\n"You can reserve an experience by opening the CultPass app, selecting your desired event, and tapping 
\'Reserve\'. Be sure to arrive 15 minutes early with your QR code ready."',
        'Each user is entitled to 4 cultural experiences per month, which may include:\n- Art exhibitions\n- Museum
entries\n- Music concerts\n- Film screenings and more\n\nSome premium experiences may require an additional fee 
(visible in the app).\n\n**Suggested phrasing:**\n"Your CultPass subscription includes 4 curated experiences each 
month. You can choose from museums, concerts, film events and more. Premium events may have an extra cost, which is
shown during reservation."',
        'Users can manage their subscription via the app > \'My Account\' > \'Manage Plan\'.\n\n- Cancel: Effective
at the end of the billing cycle\n- Pause: Preserves user data, resumes automatically when reactivated\n- Do NOT 
offer refunds unless approved by support lead\n\n**Suggested phrasing:**\n"You can cancel or pause your 
subscription at any time via the \'My Account\' section in the CultPass app. Cancelation takes effect at the end of
your billing cycle."'
    ],
    'uris': None,
    'included': ['metadatas', 'documents'],
    'data': None,
    'metadatas': [
        {
            'title': 'How to Reserve a Spot for an Event',
            'account_id': 'cultpass',
            'article_id': 'a019341c-643a-4a17-95fe-2623a38cadaf',
            'tags': 'reservation, events, booking, attendance',
            'type': 'knowledge'
        },
        {
            'article_id': '8907406f-9f93-474e-ac48-7a910675dda9',
            'type': 'knowledge',
            'tags': 'subscription, benefits, pricing, access',
            'title': "What's Included in a CultPass Subscription",
            'account_id': 'cultpass'
        },
        {
            'account_id': 'cultpass',
            'tags': 'cancelation, pause, subscription, billing',
            'article_id': '7cf2f635-ca82-4d96-b32b-4307063eaf54',
            'type': 'knowledge',
            'title': 'How to Cancel or Pause a Subscription'
        }
    ]
}

### CultPass Collection

The `cultpass` collection contains all the experiences of CultPass Card, also vectorized for similarity search.

In [34]:
Pretty(chroma_client.get_collection("cultpass").get(limit=5))

{
    'ids': [
        'experience_a6cd9a',
        'experience_b99f9b',
        'experience_68ea3d',
        'experience_a5775f',
        'experience_fe06ac'
    ],
    'embeddings': None,
    'documents': [
        "Discover the origins and vibrant traditions of Pernambuco's Carnival.",
        'Glide across calm waters at golden hour with all gear included.',
        'Wander through colorful streets and learn about Afro-Brazilian history.',
        'Dance the night away at a traditional samba club in the Lapa arches.',
        'Take a guided trip to one of the New Seven Wonders of the World with historical context.'
    ],
    'uris': None,
    'included': ['metadatas', 'documents'],
    'data': None,
    'metadatas': [
        {'title': 'Carnival History Tour in Olinda', 'experience_id': 'a6cd9a', 'type': 'experience'},
        {'experience_id': 'b99f9b', 'title': 'Sunset Paddleboarding', 'type': 'experience'},
        {'experience_id': '68ea3d', 'title': 'Pelourinho Colonial Walk', 'type': 'experience'},
        {'type': 'experience', 'experience_id': 'a5775f', 'title': 'Samba Night at Lapa'},
        {'title': 'Christ the Redeemer Experience', 'experience_id': 'fe06ac', 'type': 'experience'}
    ]
}